In [1]:
import pandas as pd
import numpy as np
import pprint

In [2]:
# base variables
csv_files = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS"]

BASE_DIR = "~/Research/wheelchair/data/raw/Max"
OUTPUT_DIR = "~/Research/wheelchair/data/processed/"

# variables for glove types
materials = ["HYB", "PLA"]

### Helper functions and config

In [94]:
def mean_positive(x):
    positives = x[x > 0]
    if len(x) == 0:
        return np.nan
    return positives.mean()

def mean_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.mean()

def nan_min(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.min()

def nan_max(x):
    positives = x[x >0]
    if len(x) == 0:
        return np.nan
    return positives.max()  


# Building the parameter dictionary
# side is "L" or "R" for left or right sides
def build_parameter_dictionary(side):
    xy_force_params = {
        "tangential_force":"N" , 
        "radial_force": "N", 
        "axle_force": "N",
        "moment_z": "Nm" 
    }
    
    # build up a dictionary containing the parameters we want to calculate
    agg_dict = {}
    for col, unit in xy_force_params.items():
        col_name = f"{col}_{side}" 
        og_column = f"{col_name}[{unit}]"
        agg_dict[f"{col_name}_pos[{unit}]"] = (og_column, mean_positive)
        agg_dict[f"{col_name}_neg[{unit}]"] = (og_column, mean_negative)
        agg_dict[f"{col_name}_pos_peak[{unit}]"] = (og_column, nan_max)
        agg_dict[f"{col_name}_neg_peak[{unit}]"] = (og_column, nan_min)

    # add the total force as well
    col = "total_force"
    unit = "N"
    col_name = f"{col}_{side}" 
    og_column = f"{col_name}[{unit}]"
    agg_dict[f"{col_name}[{unit}]"] = (og_column, mean_positive)
    agg_dict[f"{col_name}_peak[{unit}]"] = (og_column, nan_max)

    return agg_dict


# not specific to R or L side
def get_power_parameters():
    agg_dict = {}
    col = "power_z"
    unit = "W"

    og_column = f"{col}[{unit}]"
    agg_dict[f"{col}_pos[{unit}]"] = (og_column, mean_positive)
    agg_dict[f"{col}_neg[{unit}]"] = (og_column, mean_negative)
    agg_dict[f"{col}_pos_peak[{unit}]"] = (og_column, nan_max)
    agg_dict[f"{col}_neg_peak[{unit}]"] = (og_column, nan_min)
    return agg_dict

In [112]:
# load and clean the data for a given side
def load_cycle_data(path, side):
    raw_df = pd.read_csv(path)

    # only use named columns and when the hand cycle is touching
    df = raw_df.drop(columns=[c for c in raw_df.columns if "Unnamed" in c])
    df = df[df[f'theta_cop_{side}[deg]'].notna()]

    return df

def load_cycle_data_any_side(path):
    raw_df = pd.read_csv(path)

    # only use named columns and when the hand cycle is touching
    df = raw_df.drop(columns=[c for c in raw_df.columns if "Unnamed" in c])
    df = df[df[f'theta_cop_L[deg]'].notna() | df[f'theta_cop_R[deg]'].notna() ]
    return df

# Add total_force calculation
def compute_force_columns(df, side):
    # also add the total force
    df[f'total_force_{side}[N]'] = np.sqrt(
        df[f"tangential_force_{side}[N]"]**2 +
        df[f"radial_force_{side}[N]"]**2 +
        df[f"axle_force_{side}[N]"]**2
    )
    return df


def aggregate_per_cycle(df, side):
    agg_dict = build_parameter_dictionary(side)
    return df.groupby("cycle[count]").agg(**agg_dict)


# post-aggregation averages
def get_averages_from_l_r(df):
    # with positives and negatives
    side_cols = {
        "tangential_force":"N" , 
        "radial_force": "N", 
        "axle_force": "N",
        "moment_z": "Nm"
    }

    for col, unit in side_cols.items():
        L_pos = f"{col}_L_pos[{unit}]"
        L_neg = f"{col}_L_neg[{unit}]"

        R_pos = f"{col}_R_pos[{unit}]"
        R_neg = f"{col}_R_neg[{unit}]"

        L_pos_peak = f"{col}_L_pos_peak[{unit}]"
        L_neg_peak = f"{col}_L_neg_peak[{unit}]"     

        R_pos_peak = f"{col}_R_pos_peak[{unit}]"
        R_neg_peak = f"{col}_R_neg_peak[{unit}]"

        # averages
        df[f"{col}_avg_pos[{unit}]"] = df[[L_pos, R_pos]].mean(axis=1, skipna=True)
        df[f"{col}_avg_neg[{unit}]"] = df[[L_neg, R_neg]].mean(axis=1, skipna=True)


        df[f"{col}_avg_pos_peak[{unit}]"] = df[[L_pos_peak, R_pos_peak]].mean(axis=1, skipna=True)
        df[f"{col}_avg_neg_peak[{unit}]"] = df[[L_neg_peak, R_neg_peak]].mean(axis=1, skipna=True)


    # add on the total force, that is not split into positive and negatives
    df['total_force_avg[N]'] = df[['total_force_R[N]', 'total_force_L[N]']].mean(axis=1, skipna=True)
    df['total_force_avg_peak[N]'] = df[['total_force_R_peak[N]', 'total_force_L_peak[N]']].mean(axis=1, skipna=True)

    return df

def compute_per_cycle_power(path):
    df = load_cycle_data_any_side(path)

    # manually calculate the power
    df['power_z[W]'] = df['gyro_z_R[rad/s]']*df['moment_z_R[Nm]']
    agg_dict = get_power_parameters()
    return df.groupby("cycle[count]").agg(**agg_dict)
    

In [98]:

def run_all(material, initials):
    input_file = f"{BASE_DIR}/{material}/{initials}25{material}.csv"
    output_file = f"{OUTPUT_DIR}/{material}/{initials}25{material}_per_cycle.csv"

    print(f"Processing file for {initials} with gloves {material}")    

    # left side
    df_left = load_cycle_data(input_file, "L")
    df_left = compute_force_columns(df_left, "L")
    agg_df_left = aggregate_per_cycle(df_left, "L")

    # right side
    df_right = load_cycle_data(input_file, "R")
    df_right = compute_force_columns(df_right, "R")
    agg_df_right = aggregate_per_cycle(df_right, "R")

    # power calculation
    power_df = compute_per_cycle_power(input_file)
    
    # put it all together
    full_df = pd.concat([[agg_df_left, agg_df_right, power_df]], axis=1)

    result_df = get_averages_from_l_r(full_df)

    return result_df

In [132]:
# test cell
material = "PLA"
initials = "ALP"
side = "L"
input_file = f"{BASE_DIR}/{material}/{initials}25{material}.csv"
output_file = f"{OUTPUT_DIR}/{material}/{initials}25{material}_per_cycle.csv"


# left side
df_left = load_cycle_data(input_file, "L")
df_left = compute_force_columns(df_left, "L")
agg_df_left = aggregate_per_cycle(df_left, "L")

# # right side
df_right = load_cycle_data(input_file, "R")
df_right = compute_force_columns(df_right, "R")
agg_df_right = aggregate_per_cycle(df_right, "R")

# # power calculation
power_df = compute_per_cycle_power(input_file)

# # put it all together
full_df = pd.concat([agg_df_left, agg_df_right, power_df], axis=1)

result_df = get_averages_from_l_r(full_df)


In [143]:
result_df[["tangential_force_L_pos_peak[N]", "tangential_force_R_pos_peak[N]", "tangential_force_avg_pos_peak[N]"]]

,tangential_force_L_pos_peak[N],tangential_force_R_pos_peak[N],tangential_force_avg_pos_peak[N]
cycle[count],,,
1.0,223.197050,212.614120,217.905585
2.0,240.967864,231.880218,236.424041
3.0,266.850079,255.579805,261.214942
4.0,253.822253,273.137061,263.479657
5.0,267.495659,265.972705,266.734182
6.0,230.210486,261.492807,245.851646
7.0,240.160030,245.085437,242.622733
8.0,223.450598,204.961172,214.205885
9.0,203.029858,227.786530,215.408194


In [105]:
agg_df_right.head()

,tangential_force_R_pos[N],tangential_force_R_neg[N],tangential_force_R_pos_peak[N],tangential_force_R_neg_peak[N],radial_force_R_pos[N],radial_force_R_neg[N],radial_force_R_pos_peak[N],radial_force_R_neg_peak[N],axle_force_R_pos[N],axle_force_R_neg[N],axle_force_R_pos_peak[N],axle_force_R_neg_peak[N],moment_z_R_pos[Nm],moment_z_R_neg[Nm],moment_z_R_pos_peak[Nm],moment_z_R_neg_peak[Nm],total_force_R[N],total_force_R_peak[N]
cycle[count],,,,,,,,,,,,,,,,,,
1.0,133.476773,-6.063134,212.614120,-7.686505,63.778973,-8.479698,150.863353,-14.148697,119.211070,NaN,204.107296,NaN,24.110727,-1.230439,37.198151,-1.841134,187.260035,276.335014
2.0,138.241672,-9.236256,231.880218,-12.927013,106.185599,-38.889029,193.554375,-72.153681,117.112869,NaN,186.127209,NaN,24.648236,-1.500566,40.360197,-2.175886,200.164798,312.354937
3.0,164.435854,-6.384256,255.579805,-6.384256,125.428622,-49.230529,214.311780,-76.736938,137.046056,NaN,196.449111,NaN,27.584305,-3.302280,42.477276,-3.302280,241.270286,357.314189
4.0,155.266577,-8.635437,273.137061,-13.943038,139.489649,-25.161552,234.369536,-52.011231,133.779198,NaN,222.287164,NaN,25.586067,-2.016050,40.776375,-5.500785,233.056946,398.620682
5.0,155.224664,-8.537879,265.972705,-27.640296,132.391070,-32.663714,250.639259,-56.786873,125.207118,NaN,235.872119,NaN,24.232510,-1.812054,37.220769,-6.152193,224.991145,408.607062
